In [4]:
#sudo pip install ibm_db
import ibm_db
# conda install -c conda-forge ibm_db

import pandas as pd
import numpy as np
import psycopg2
# conda install -c anaconda psycopg2
import sqlalchemy
# conda install -c anaconda sqlalchemy
import matplotlib as plt
import requests
# conda install -c anaconda requests
from requests.auth import HTTPDigestAuth
import json
# conda install -c jmcmurray json
import urllib.request
import urllib.parse
# conda install -c anaconda zlib
import sys
import asyncio

from sqlalchemy import create_engine

import ibm_db_sa
# conda install -c auto ibm_db_sa
# pip install ibm-db-sa

import ibm_db_dbi

import logging

import datetime

from my_functions import *

In [5]:
logging.basicConfig(filename='logs.log',level=logging.DEBUG)


POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'ebreshna_user'
POSTGRES_PASSWORD = 'secret'
POSTGRES_DBNAME = 'ebreshna_test'
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))
# conn = psycopg2.connect("host=localhost dbname=asims_test user=asims_user password=secret port=5432")
# cur = conn.cursor()
psql_engine = create_engine(postgres_str)

# SQL SERVER CONNECTION
DB2_ADDRESS = '10.10.100.47'
DB2_PORT = '53000'
DB2_USERNAME = 'db2inst1'
DB2_PASSWORD = 'db2inst1'
DB2_DBNAME = 'mpower'
# db2_engine = create_engine('ibm_db_sa+pyodbc:///?odbc_connect={}'.format(quoted))
# db2_engine = create_engine("db2+ibm_db://user:pass@10.10.100.47:51000/mpower")
# conn = ibm_db.connect("DATABASE=mpower;HOSTNAME=host;PORT=50000;PROTOCOL=TCPIP;UID=username;PWD=password;", "", "")
db2_conn = ibm_db.connect("DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;", "", "")
# this is required to enable database access through Pandas
db2_conni =  ibm_db_dbi.connect(u"DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;")

In [6]:
# Region
cols_map = [
    {'src_col': 'ID', 'des_col': 'id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REGISTRATION_NUMBER', 'des_col': 'registration_number', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'STATUS_ID', 'des_col': 'status_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OM_NO', 'des_col': 'om_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OM_DT', 'des_col': 'om_dt', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'REMARKS', 'des_col': 'remarks', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RESULT_DESC', 'des_col': 'result_desc', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RESULT_NO', 'des_col': 'result_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RESULT_DT', 'des_col': 'result_dt', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'PROCESSED_BY', 'des_col': 'processed_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PROCESSED_DESIG', 'des_col': 'processed_desig', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PRCOESSED_ON', 'des_col': 'prcoessed_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'OVERHEAD_CHARGES1', 'des_col': 'overhead_charges1', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'OVERHEAD_CHARGES2', 'des_col': 'overhead_charges2', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'SECURITY_DEPOSIT', 'des_col': 'security_deposit', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'SERVICE_LINE_CHARGES', 'des_col': 'service_line_charges', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'SYSTEM_LOADING_CHARGES', 'des_col': 'system_loading_charges', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PROCESSING_FEE', 'des_col': 'processing_fee', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'METER_COST', 'des_col': 'meter_cost', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'METER_TESTING_FEE', 'des_col': 'meter_testing_fee', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'RECON_DISCON_FEE', 'des_col': 'recon_discon_fee', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PD_FEE', 'des_col': 'pd_fee', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'OTHER_FEE', 'des_col': 'other_fee', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'SERVICE_TAX', 'des_col': 'service_tax', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'EDUCATION_CESS', 'des_col': 'education_cess', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'OTHER_FEEIFANY1', 'des_col': 'other_feeifany1', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'OTHER_FEEIFANY2', 'des_col': 'other_feeifany1', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'OTHER_FEEIFANY3', 'des_col': 'other_feeifany1', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'CREATED_BY', 'des_col': 'created_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CREATE_DATE', 'des_col': 'created_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'UPDATED_BY', 'des_col': 'updated_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'UPDATE_DATE', 'des_col': 'updated_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'RECORD_STATUS', 'des_col': 'record_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DB_IP_ADDRESS', 'des_col': 'db_ip_address', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SOURCE_OFFICE_ID', 'des_col': 'source_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TARGET_OFFICE_ID', 'des_col': 'target_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SCREEN_ID', 'des_col': 'screen_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MODULE_ID', 'des_col': 'module_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUBMODULE_ID', 'des_col': 'submodule_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG1', 'des_col': 'flag1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG2', 'des_col': 'flag2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG3', 'des_col': 'flag3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG4', 'des_col': 'flag4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG5', 'des_col': 'flag5', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FOLLOWUP_TIME', 'des_col': 'followup_time', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'CHECK_CONDITION', 'des_col': 'check_condition', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REASON_FOR_PENDING', 'des_col': 'reason_for_pending', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PENDING_AT', 'des_col': 'pending_at', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PENDING_PERSON', 'des_col': 'pending_person', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ESCALATION_AUTHORITY', 'des_col': 'escalation_authority', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ESCALATION_OFFICE', 'des_col': 'escalation_office', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ESCALATION_DETAILS', 'des_col': 'escalation_details', 'src_dtype': 'String', 'des_dtype': 'String'}
]

migrated_row_count = migrate_data(db2_conni, psql_engine, 'DABS    .NCT_HTREGISTRATION_MONITOR', 'public', 'nct_htregistration_monitor', cols_map)
print("Total Migrated Records (nct_htregistration_monitor): " + str(migrated_row_count))


Total Migrated Records (nct_htregistration_monitor): Total Src Records: 0 Total migrated Records to dest: 0
